In [200]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

### Extract CSVs into DataFrames

In [201]:
#load file into panda. make sure to add thousands=',' in read_csv as some of the figures contain commas
cocoa_daily = "Resources/Cocoa Prices - Daily Prices 1970-2020.csv"
cocoa_monthly = "Resources/Cocoa Prices - Monthly Average 1970-2020.csv"
cocoa_prod_indices="Resources/production_indices.csv"
cocoa_daily_df = pd.read_csv(cocoa_daily, thousands=',')
cocoa_monthly_df = pd.read_csv(cocoa_monthly, thousands=',')
cocoa_prod_indices_df=pd.read_csv(cocoa_prod_indices, thousands=',')
value_ag_df = pd.read_csv("Resources/value_agriculture.csv")
trade_df = pd.read_csv("Resources/detailed_trade_matrix.csv")
util_csv = "Resources/supply_utilisation.csv"
rating_csv = "Resources/chocolate_rating.csv"
filename = 'Resources/crop_livestock_production.csv'

<h1>Transform premise DataFrame</h1>

<h2>Country By Region Table</h2>

In [202]:
url = 'https://meta.wikimedia.org/wiki/List_of_countries_by_regional_classification'
tables = pd.read_html(url)
country_region = tables[0]

In [203]:
country_region = country_region.drop("Global South", axis = 1)

In [204]:
# country_regions.to_csv('country_region.csv', index=False)

<h2>Crop Livestock Production Table</h2>

In [205]:
area_data = pd.read_csv(filename)
columns = area_data.columns

In [206]:
production = area_data.drop(['Domain Code', 'Domain', 'Area Code', 'Element Code',
       'Item Code', 'Item', 'Year Code', 'Flag',
       'Flag Description'], axis = 1)

In [207]:
# Split production table into 3 tables, one for each Element

# first table
area_harvested = production.loc[production['Element'] == "Area harvested"]

In [208]:
# drop unnecessary column
area_harvested = area_harvested.drop('Element', axis = 1)

In [209]:
# re-label columns
area_harvested.columns = ['Country', 'Year', 'Area_unit', 'Area_harvested']

In [210]:
# repeat for other columns
prod = production.loc[production['Element'] == "Production"]
prod = prod.drop('Element', axis = 1)
prod.columns = ['Country', 'Year', 'Production_unit', 'Production']

p_yield = production.loc[production['Element'] == "Yield"]
p_yield = p_yield.drop('Element', axis = 1)
p_yield.columns = ['Country', 'Year', 'Yield_unit', 'Yield']

In [211]:
# combine tables
df1 = pd.merge(area_harvested, prod, how = "outer", on=["Country", "Year"])
country_production = pd.merge(df1, p_yield, how = "outer", on=["Country", "Year"])

In [212]:
# # export dataframe so it can be imported into sql
# country_production.to_csv("country_production.csv")

In [213]:
# merge the two tables to get the region
production_country_region = pd.merge(country_production, country_region, how = "inner", left_on = "Country", right_on = "Country")

In [214]:
# drop the Country column
production_country_region = production_country_region.drop("Country", axis = 1)

In [215]:
production_country_region.columns

Index(['Year', 'Area_unit', 'Area_harvested', 'Production_unit', 'Production',
       'Yield_unit', 'Yield', 'Region'],
      dtype='object')

In [216]:
# some columns need to be summed and others averaged
calc = {'Area_unit':'first', 'Area_harvested':sum, 'Production_unit':'first',
       'Production':sum, 'Yield_unit':'first', 'Yield':'mean' }
production_region = production_country_region.groupby(["Region", "Year"]).agg(calc)

In [217]:
# reset index so table is easy to export and load
production_region = production_region.reset_index()

In [218]:
# # export table
# production_region.to_csv('region_production.csv', index=False)

<h2>Agriculture Value Table</h2>

In [219]:
value_columns = ["Area", "Element", "Year", "Unit", "Value"]
new_value_ag_df = value_ag_df[value_columns].copy()

In [220]:
new_value_ag_df["Element"] = new_value_ag_df["Element"].str[:22]

In [221]:
## the below csv has been saved under Mel code folder
# new_value_ag_df.to_csv("Resources.cleaned_value_ag.csv")

<h2><strong>Trade Matrix</strong></h2>

In [222]:
trade_columns = ["Reporter Countries", "Partner Countries", "Element", "Year", "Unit", "Value"]
new_trade_df = trade_df[trade_columns].copy()

In [223]:
new_trade_df = new_trade_df.rename(columns={"Reporter Countries": "Reporter_countries",
                                     "Partner Countries": "Partner_countries"})

In [224]:
# split into 4 different elements
import_qty = new_trade_df.loc[new_trade_df["Element"] == "Import Quantity"]
import_value = new_trade_df.loc[new_trade_df["Element"] == "Import Value"]
export_qty = new_trade_df.loc[new_trade_df["Element"] == "Export Quantity"]
export_value = new_trade_df.loc[new_trade_df["Element"] == "Export Value"]

In [225]:
# drop zero values
import_qty = import_qty[import_qty["Value"] != 0]
import_value = import_value[import_value["Value"] != 0]
export_qty = export_qty[export_qty["Value"] != 0]
export_value = export_value[export_value["Value"] != 0]

In [226]:
# merge tables
df1 = pd.merge(import_qty, import_value, how="outer", on=["Reporter_countries", "Partner_countries", "Year"])
df2 = pd.merge(export_qty, export_value, how="outer", on=["Reporter_countries", "Partner_countries", "Year"])
df_3 = pd.merge(df1,df2, how="outer", on=["Reporter_countries", "Partner_countries", "Year"])

In [227]:
df_3.dropna(inplace=True)
df_3.head()
# rename columns
df_3 = df_3.rename(columns={"Element_x_x": "Import_qty",
               "Unit_x_x": "Import_qty_unit",
                "Value_x_x": "Import_qty_value",
                "Element_y_x": "Import_value",
                "Unit_y_x": "Import_value_unit",
                "Value_y_x": "Import_value_value",
                "Element_x_y": "Export_qty",
                "Unit_x_y": "Exporty_qty_unit",
                "Value_x_y": "Export_qty_value",
                "Element_y_y": "Export_value",
                "Unit_y_y": "Export_value_unit",
                "Value_y_y": "Export_value_value"})

In [228]:
# new_trade_df.to_csv("Resources.cleaned_trade_matrix.csv")

<h2><strong>Supply Utilisation & Chocolate Rating</strong><h2>

In [229]:
util_df = pd.read_csv(util_csv)
util_df = util_df[util_df['Value'] !=0]

In [230]:
rating_df = pd.read_csv(rating_csv)

In [231]:
df = util_df[util_df['Element'] == 'Food supply quantity (tonnes)']

In [232]:
util_df_cols = ['Area', 'Element', 'Year', 'Unit', 'Value']
Utilisation = df[util_df_cols].copy()
Utilisation.sort_values(['Year'], ascending=False)

,Area,Element,Year,Unit,Value
2615,Kuwait,Food supply quantity (tonnes),2018,tonnes,2
771,Cameroon,Food supply quantity (tonnes),2018,tonnes,9533
1088,Comoros,Food supply quantity (tonnes),2018,tonnes,13
1407,Dominica,Food supply quantity (tonnes),2018,tonnes,50
1979,Grenada,Food supply quantity (tonnes),2018,tonnes,200
...,...,...,...,...,...
2373,Ireland,Food supply quantity (tonnes),2014,tonnes,4
3999,Saint Lucia,Food supply quantity (tonnes),2014,tonnes,40
3623,Papua New Guinea,Food supply quantity (tonnes),2014,tonnes,4001
3393,Niger,Food supply quantity (tonnes),2014,tonnes,3


In [233]:
Utilisation = Utilisation.rename(columns={'Area':'area', 'Element':'element',
                                'Year':'year', 'Unit':'unit', 'Value':'value'})

In [234]:
 rating_cols = ['company', 'company_location', 'review_date', 'cocoa_percent', 'rating']
 rating_cols2 = rating_df[rating_cols].copy()

In [235]:
Choc_Rating = rating_cols2.rename(columns={'company_location':'area',
                                    'review_date':'year',
})

<h2><strong>Cocoa Daily Price</strong></h2>
- clean up cocoa_daily_df

In [236]:
# filter out cocoa_daily_df to only include prices in USD/tonnes
cocoa_daily_df=cocoa_daily_df[['Date','New York futures (US$/tonne)', 'ICCO daily price (US$/tonne)']]

In [237]:
#add 'year' and 'month/year' column in cocoa_daily_df 
cocoa_daily_df['year_code']=cocoa_daily_df['Date'].str[-4:]
cocoa_daily_df['month_code']=cocoa_daily_df['Date'].str[-7:]

cocoa_daily_df=cocoa_daily_df.rename(columns={'New York futures (US$/tonne)':"New_York_futures_USD/tonne", 'ICCO daily price (US$/tonne)':"ICCO_USD/tonne"})
cocoa_daily_df.head()

# # the above will allow analysts to create another table to summarise cocoa prices on yearly & monthly basis (i.e average price, max, and min for each year)
# sample code
# cocoa_price_yearly_summary=round(cocoa_daily_df.groupby('year').agg({'New York futures (US$/tonne)' : ['mean', 'min', 'max'], 'ICCO daily price (US$/tonne)' : ['mean', 'min', 'max']}),2)
# cocoa_price_monthly_summary=round(cocoa_daily_df.groupby('month/year').agg({'New York futures (US$/tonne)' : ['mean', 'min', 'max'], 'ICCO daily price (US$/tonne)' : ['mean', 'min', 'max']}),2)


,Date,New_York_futures_USD/tonne,ICCO_USD/tonne,year_code,month_code
0,31/12/2020,2544.33,2424.35,2020,12/2020
1,30/12/2020,2534.00,2412.99,2020,12/2020
2,29/12/2020,2459.33,2334.66,2020,12/2020
3,28/12/2020,2502.00,2355.17,2020,12/2020
4,24/12/2020,2537.33,2385.85,2020,12/2020


In [238]:
#create a table to show yearly average price, min price, & max price recorded in New York Futures
cocoa_NYF=cocoa_daily_df.copy()
del cocoa_NYF['ICCO_USD/tonne']
cocoa_NYF_yearly= round(cocoa_NYF.groupby('year_code').agg(['mean', 'min', 'max']),2)
cocoa_NYF_yearly.columns = cocoa_NYF_yearly.columns.droplevel()

cocoa_NYF_yearly.head()

,mean,min,max
year_code,,,
1994,1324.02,1237.67,1391.33
1995,1350.05,1252.00,1474.67
1996,1370.41,1242.67,1466.67
1997,1544.44,1252.67,1783.33
1998,1601.63,1400.67,1762.33


In [239]:
#create a table to show yearly average price, min price, & max price recorded in ICCO
cocoa_ICCO=cocoa_daily_df.copy()
del cocoa_ICCO['New_York_futures_USD/tonne']
cocoa_ICCO_yearly= round(cocoa_ICCO.groupby('year_code').agg(['mean', 'min', 'max']),2)
cocoa_ICCO_yearly.columns = cocoa_ICCO_yearly.columns.droplevel()

cocoa_ICCO_yearly.head()

,mean,min,max
year_code,,,
1994,1428.25,1346.41,1497.14
1995,1433.32,1335.09,1568.62
1996,1455.81,1313.15,1572.89
1997,1620.86,1322.59,1834.68
1998,1675.51,1462.83,1830.04


In [240]:
#create a table to show monthly average price, min price, & max price recorded in New York Futures
cocoa_NYF_monthly= round(cocoa_NYF.groupby('month_code').agg(['mean', 'min', 'max']),2)
cocoa_NYF_monthly.columns = cocoa_NYF_monthly.columns.droplevel()

cocoa_NYF_monthly.head()

,mean,min,max
month_code,,,
01/1995,1368.54,1305.33,1408.67
01/1996,1287.65,1265.33,1310.33
01/1997,1361.29,1293.00,1415.67
01/1998,1612.79,1570.33,1667.00
01/1999,1383.05,1337.00,1436.67


In [241]:
#create a table to show monthly average price, min price, & max price recorded in ICCO
cocoa_ICCO_monthly= round(cocoa_ICCO.groupby('month_code').agg(['mean', 'min', 'max']),2)
cocoa_ICCO_monthly.columns = cocoa_ICCO_monthly.columns.droplevel()

cocoa_ICCO_monthly.head()

,mean,min,max
month_code,,,
01/1995,1468.82,1411.14,1504.70
01/1996,1349.54,1331.62,1370.44
01/1997,1428.48,1365.20,1481.16
01/1998,1668.15,1624.06,1722.74
01/1999,1455.23,1417.87,1501.53


<strong> Production Indices </strong>
- based on http://www.fao.org/waicent/faostat/agricult/indices-e.htm, The FAO indices of agricultural production show the relative level of the aggregate volume of agricultural production for each year in comparison with the base period 1999-2001. They are based on the sum of price-weighted quantities of different agricultural commodities produced after deductions of quantities used as seed and feed weighted in a similar manner. The resulting aggregate represents, therefore, disposable production for any use except as seed and feed.
- clean up Production Indices table

In [242]:
prod_indices_df= cocoa_prod_indices_df[['Domain', 'Area Code', 'Area', 'Element', 'Item', 'Year', 'Unit', 'Value']]
prod_indices_df=prod_indices_df.rename(columns={'Area Code': 'area_code'})
prod_indices_df.head()

,Domain,area_code,Area,Element,Item,Year,Unit,Value
0,Production Indices,7,Angola,Gross Production Index Number (2014-2016 = 100),"Cocoa, beans",1961,index,92.74
1,Production Indices,7,Angola,Gross Production Index Number (2014-2016 = 100),"Cocoa, beans",1962,index,92.74
2,Production Indices,7,Angola,Gross Production Index Number (2014-2016 = 100),"Cocoa, beans",1963,index,69.55
3,Production Indices,7,Angola,Gross Production Index Number (2014-2016 = 100),"Cocoa, beans",1964,index,69.55
4,Production Indices,7,Angola,Gross Production Index Number (2014-2016 = 100),"Cocoa, beans",1965,index,92.74


<h1>Create database connection</h1>

List of Dataframe to be imported:
- cocoa_daily_df
- cocoa_monthly_df
- cocoa_NYF_yearly
- cocoa_ICCO_yearly
- cocoa_NYF_monthly
- cocoa_ICCO_monthly
- prod_indices_df

In [243]:
connection_string = f"{username}:{password}@localhost:5432/ETL_project"
engine = create_engine(f'postgresql://{connection_string}')

In [244]:
# Confirm tables
engine.table_names()

['detailed_trade_matrix',
 'value_agriculture',
 'cocoa_daily_prices',
 'cocoa_monthly_prices',
 'NYF_yearly_summary',
 'ICCO_yearly_summary',
 'NYF_monthly_summary',
 'ICCO_monthly_summary',
 'production_indices',
 'country_region',
 'utilisation',
 'choc_rating']

<h1>Load DataFrames into database</h1>

In [253]:
cocoa_daily_df.to_sql(name='cocoa_daily_prices', con=engine, if_exists='append', index=False)

In [254]:
cocoa_monthly_df.to_sql(name='cocoa_monthly_prices', con=engine, if_exists='append', index=False)

In [256]:
cocoa_NYF_yearly.to_sql(name='NYF_yearly_summary', con=engine, if_exists='append', index=True)

In [257]:
cocoa_NYF_monthly.to_sql(name='NYF_monthly_summary', con=engine, if_exists='append', index=True)

In [259]:
cocoa_ICCO_yearly.to_sql(name='ICCO_yearly_summary', con=engine, if_exists='append', index=True)

In [260]:
cocoa_ICCO_monthly.to_sql(name='ICCO_monthly_summary', con=engine, if_exists='append', index=True)

In [261]:
prod_indices_df.to_sql(name='production_indices', con=engine, if_exists='append', index=False)

In [262]:
country_regions.to_sql(name="country_region", con=engine, if_exists='append', index=False)

In [263]:
new_value_ag_df.to_sql(name='value_agriculture', con=engine, if_exists='append', index=False)

In [264]:
new_trade_df.to_sql(name='detailed_trade_matrix', con=engine, if_exists='append', index=False)

In [265]:
Utilisation.to_sql(name="utilisation", con=engine, if_exists='append', index=False)

In [266]:
Choc_Rating.to_sql(name="choc_rating", con=engine, if_exists='append', index=False)

In [268]:
country_production.to_sql(name="country_production", con=engine, if_exists='append', index=False)